Zofia Maszlanka s22555 <br>
Tomasz Latała s23864

In [1]:
!python -m pip install -U spacy
!python -m spacy download pl_core_news_md
!python -m spacy download pl_core_news_sm

     |████████████████████████████████| 87.9 MB 21 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_md')
     |████████████████████████████████| 58.6 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [2]:
!python -m pip install emoji


import pandas as pd
import numpy as np
import string
import spacy
import re
import emoji
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split




In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/tweets.csv'

In [5]:
dataset = pd.read_csv(path, sep='\t', header = None, names = ['tweet'])

In [6]:
dataset.head(100)

,tweet
0,Podejrzenie wirusa Coronavirus https://t.co/Zb...
1,Boję się tego wirusa hshshshshs #coronavirus
2,Pierwszy przypadek wirusa #2019nCoV w Kaliforn...
3,"Kiedy #coronavirus dotrze do Polski, napotka o..."
4,Prawdopodobnie nie uchroni od #coronavirus ale...
...,...
95,@SzumowskiLukasz może być potrzebne zawierzeni...
96,https://t.co/yCUbccUcmH Nie trzeba #coronaviru...
97,Sanepid o koronawirusie: nie ma powodów do pan...
98,"Forma wyrażania gniewu, protestu... Do Wuhan p..."


In [7]:
len(dataset)

307692

In [8]:
def preprocess(text):
 
  text = text.str.lower()

  nlp = spacy.load('pl_core_news_md')
  stopwordlist = nlp.Defaults.stop_words
  def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwordlist])

  punctuations_list = string.punctuation
  def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list) 
    return text.translate(translator)

  to_remove = [' rt ', 'icymi', 'wuhan', 'koronawirus ', 'wirus ', 'coronavirus '] #means retweet, in case you missed it
  def cleaning_xdd(text):
    return re.sub(r'x(d)\1+', r'', text)

  def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)

  def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)

  def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

  def cleaning_emojis(text):
    return emoji.get_emoji_regexp().sub(u'', text)

  def cleaning_usernames(data):
    return re.sub('@(.*?)($|\s)', '', data)


  text = text.map(lambda x: cleaning_stopwords(x)) 
  text = text.map(lambda x: cleaning_xdd(x))
  for word in to_remove:
    text = text.map(lambda x: x.replace(word, ''))
  text = text.map(lambda x: cleaning_repeating_char(x))
  text = text.map(lambda x: cleaning_URLs(x))
  text = text.map(lambda x: cleaning_numbers(x))
  text = text.map(lambda x: cleaning_emojis(x))
  text = text.map(lambda x: cleaning_usernames(x))
  text = text.map(lambda x: cleaning_punctuations(x))
 
  return(text)




In [9]:
dataset['tweet'] = preprocess(dataset['tweet'])

In [10]:
nlpPL = spacy.load('pl_core_news_sm')
def lemmatization(data):
  text = nlpPL(data)
  return(" ".join([token.lemma_ for token in text]))

In [ ]:
dataset['tweet'] = dataset['tweet'].apply(lambda x: lemmatization(x))
dataset.head(10)

In [ ]:
to_remove = ['koronawirus', 'wirus', 'coronavirus'] 
for word in to_remove:
  dataset['tweet'] = dataset['tweet'].map(lambda x: x.replace(word, ''))
dataset.dropna()
dataset.drop_duplicates()

In [ ]:
len(dataset)

In [ ]:
dataset.head(100)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) #unigramy i bigramy
vectorizer.fit(dataset['tweet'])
print('Featured words: ', len(vectorizer.get_feature_names()))

In [ ]:
dataset_v = vectorizer.transform(dataset['tweet'])

In [ ]:
print(dataset_v.shape)
print(dataset.shape)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(dataset_v)
pred = kmeans.predict(dataset_v)


In [ ]:
unique, counts = np.unique(pred, return_counts=True)
for i in range (3):
  print(unique[i], " : ", counts[i])

In [ ]:
df = pd.DataFrame(pred)
dataset = dataset.join(df)
dataset0 = dataset[dataset[0] == 0]
dataset1 = dataset[dataset[0] == 1]
dataset2 = dataset[dataset[0] == 2]

In [ ]:
dataset0.head(100)

In [ ]:
dataset1.head(100)

In [ ]:
dataset2.head(100)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['tweet'], dataset[0], test_size = 0.2, random_state =42)

In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2)) 
vectorizer.fit(X_train)
def vectorize(x):
  return vectorizer.transform(x)

In [ ]:
X_train_v = vectorize(X_train)
X_test_v = vectorize(X_test)

In [ ]:
def model_Evaluate(model):
  
  y_pred = model.predict(X_test_v)

  print(classification_report(y_test, y_pred))

  cf_matrix = confusion_matrix(y_test, y_pred)
  categories = ['Negative','Positive']
  group_names = ['True Neg','False Pos', 'False Neg','True Pos']
  group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
  labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names, group_percentages)]
  labels = np.asarray(labels).reshape(2,2)
  sns.heatmap(cf_matrix, cmap = 'Blues',fmt = '', annot = True)
  plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
  plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
  plt.title("Confusion Matrix", fontdict = {'size':18}, pad = 20)

# **SVC**

In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(X_train_v, y_train)
model_Evaluate(SVCmodel)
y_pred2 = SVCmodel.predict(X_test_v)

# **LRmodel**

In [ ]:
LRmodel = LogisticRegression(max_iter = 1000)
LRmodel.fit(X_train_v, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test_v)

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train_v, y_train)
model_Evaluate(rf_classifier)
y_pred_rf = rf_classifier.predict(X_test_v)